## SF crime data analysis and modeling

### Use Spark SQL for big data analysis on SF crime data. 
(https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data).

In [3]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from ggplot import *
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [4]:
#download data from SF gov's official website
import urllib.request
urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))

#or download the file locally
#https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD


path,name,size
dbfs:/laioffer/spark_hw1/data/sf_03_18.csv,sf_03_18.csv,544214906


In [5]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"


In [6]:
#Read data from the data storage
#Split the header by its separator
crime_data_lines = sc.textFile(data_path)
#prepare data: remove "
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])
#get header
header = df_crimes.first()
print(header)

#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

#get the first three lines of data
#display(crimes.take(3))

#get the total number of data 
print(crimes.count())


['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId', 'SF Find Neighborhoods', 'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods', ':@computed_region_yftq_j783', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py', ':@computed_region_4isq_27mq', ':@computed_region_fcz8_est8', ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7', ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi', ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4']
2215024

### Solove  big data issues via Spark
approach: use SQL

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#Load .csv
df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)

#display data 
#display(df_opt1)

#create temp table
df_opt1.createOrReplaceTempView("sf_crime")

#display first 10 rows
first_ten_rows = spark.sql("SELECT * FROM sf_crime LIMIT 10")
display(first_ten_rows)


IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_yftq_j783,:@computed_region_p5aj_wyqh,:@computed_region_rxqg_mtj9,:@computed_region_bh8s_q3mv,:@computed_region_fyvs_ahh9,:@computed_region_9dfj_4gjx,:@computed_region_n4xg_c4py,:@computed_region_4isq_27mq,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6pnf_4xz7,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4
180362289,VEHICLE THEFT,STOLEN MOTORCYCLE,Tuesday,05/15/2018,10:30,SOUTHERN,NONE,700 Block of TEHAMA ST,-122.41191202732877,37.77520656149669,"(37.77520656149669, -122.41191202732877)",18036228907023,32,1,10,34,8,2,9,28853,34,null,1,null,1,null,null,2,null,null,1,null
180360948,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,05/15/2018,04:14,SOUTHERN,NONE,MARKET ST / SOUTH VAN NESS AV,-122.41925789481357,37.77514629165388,"(37.77514629165388, -122.41925789481357)",18036094864020,32,1,10,20,8,2,9,28853,19,null,1,null,1,null,8,2,1,1,1,null
180360879,OTHER OFFENSES,PAROLE VIOLATION,Tuesday,05/15/2018,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087926150,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null
180360879,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Tuesday,05/15/2018,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087965010,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null
180360879,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,05/15/2018,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087965015,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null
180360829,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Tuesday,05/15/2018,01:27,MISSION,NONE,700 Block of SHOTWELL ST,-122.41561725232026,37.75641376904809,"(37.75641376904809, -122.41561725232026)",18036082965016,53,3,2,20,2,4,7,28859,19,null,null,null,3,null,null,2,null,null,3,null
180360835,ROBBERY,"ROBBERY, BODILY FORCE",Tuesday,05/15/2018,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.41004163181597,37.781953653725715,"(37.781953653725715, -122.41004163181597)",18036083503074,32,5,10,34,14,2,9,28853,34,17,1,18,1,18,7,2,1,1,1,null
180360835,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Tuesday,05/15/2018,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.41004163181597,37.781953653725715,"(37.781953653725715, -122.41004163181597)",18036083516710,32,5,10,34,14,2,9,28853,34,17,1,18,1,18,7,2,1,1,1,null
180360794,LIQUOR LAWS,MISCELLANEOUS LIQOUR LAW VIOLATION,Tuesday,05/15/2018,00:19,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.44776112231955,37.76984648754153,"(37.76984648754153, -122.44776112231955)",18036079417030,25,7,11,3,15,5,11,29492,9,22,null,24,null,25,null,1,null,null,null,null
180360794,WARRANTS,ENROUTE TO OUTSIDE JURISDICTION,Tuesday,05/15/2018,00:19,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.44776112231955,37.76984648754153,"(37.76984648754153, -122.44776112231955)",18036079462050,25,7,11,3,15,5,11,29492,9,22,null,24,null,25,null,1,null,null,null,null


### Q1 (OLAP): 
##### Count the number of crimes for different categories:

In [10]:
q1_result = df_opt1.groupBy('category').count().orderBy('count', ascending=False)

#display result
display(q1_result)

category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


In [11]:
#Spark SQL based
##in descending order
crimeCategory_desc = spark.sql("SELECT category, COUNT(*) AS count FROM sf_crime GROUP BY category ORDER BY Count DESC")

#display result
display(crimeCategory_desc)

##in ascecding order
#crimeCategory_asc = spark.sql("SELECT category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count ASC")
#display(crimeCategory_asc)

category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


In [12]:
#important hints: 
##first step: spark df or sql to compute the statisitc result 
##second step: export your result to a pandas dataframe. 

##show data
#crimeCategory_desc.show()

##convert to pandas dataframe
crimes_pd_df = crimeCategory_desc.toPandas()

##Spark does not support this function, please refer https://matplotlib.org/ for visuliation. You need to use display to show the figure in the databricks community. 

##display/visualize the result to a pandas dataframe
display(crimes_pd_df)


category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


#### Q1. Conclusion:
Larceny/Theft category has the highest count, meaning that this type of crime happens the most in SF, whereas trea has the lowest count, trea is observed the least time.

### Q2 (OLAP)
Count the number of crimes for different district, and visualize your results:

In [15]:
#Spark SQL based - Soulution for Q2
##count the number of crimes for different district using column PdDistrict

##in descending order
crimePdDistrict_desc = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count DESC")

#display result
display(crimePdDistrict_desc)

##in ascending order
#crimePdDistrict_asc = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count ASC")
#display(crimePdDistrict_asc)

PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


In [16]:
#visualize the result
#pandas dateframe

crimes2_pd_df = crimePdDistrict_desc.toPandas()

#display result
display(crimes2_pd_df)


PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


#### Q2. Conclustion:
Southern PdDistrict has the highest number of crimes, whereas crimes in Richmond are low,
we can conclude that Richmond Park, and Taraval are the top3 safest districts, Southern district is the most dangerous district.

### Q3 (OLAP)
Count the number of crimes each "Sunday" at "SF downtown".   
Used python user-defined function

In [19]:
##google search appears SF downtown (financial district)'s coordinates to be: 37.7946° N, 122.3999° W
##therefore, we can assume SF downtown is centered at 37.7946° N, 122.3999° W
##within a range of 1.19 square kilometers (approx. 119ha) is called 'SF downtown' / largest distance to the center point: (1.19/3.14)**0.5 = 0.616 km
##assume 1° ≈ 111 km at longitude & latitude

#obtain longitude & latitude
crimesSFdt_x = spark.sql("SELECT DayOfWeek, Date, Address, X FROM sf_crime")
crimesSFdt_y = spark.sql("SELECT DayOfWeek, Date, Address, Y FROM sf_crime")
#crimesSFdt_x.show()
#crimesSFdt_y.show()

from pyspark.sql.types import FloatType

def x_square_float(x):
    return ((float(x)-(-122.3999))*111)**2 

x_square_udf_float = udf(lambda z: x_square_float(z), FloatType())
crimes3_df = crimesSFdt_x.select('DayOfWeek', 'Date', 'Address', 'X', x_square_udf_float('X').alias('x_float_squared'))
#crimes3_df.show()

#convert to Pandas dataframe
#crimes3_pd_df = crimes3_df.toPandas()
#display(crimes3_pd_df)

def y_square_float(y):
    return ((float(y)-37.7946)*111)**2 

y_square_udf_float = udf(lambda z: y_square_float(z), FloatType())
crimes4_df = crimesSFdt_y.select('DayOfWeek', 'Date', 'Address', 'Y', y_square_udf_float('Y').alias('y_float_squared'))
#crimes4_df.show()

#convert to Pandas dataframe
#crimes4_pd_df = crimes4_df.toPandas()
#display(crimes4_pd_df)

#Create temporary table called df3, df4
crimes3_df.createOrReplaceTempView("df3")
crimes4_df.createOrReplaceTempView("df4")

#write spark.SQL to retrieve distance
crimesSFdt = spark.sql("SELECT dt.Date, COUNT(distance) AS num_of_crimes \
                       FROM ( \
                         SELECT df3.DayOfWeek, df3.Date, df3.Address, SQRT(df3.x_float_squared + df4.y_float_squared) as distance \
                         FROM df3 \
                         INNER JOIN df4 \
                         ON df3.address = df4.address \
                         WHERE df3.DayOfWeek = 'Sunday' \
                       ) AS dt \
                       WHERE dt.distance < 0.616 \
                       GROUP BY dt.Date \
                       ORDER BY num_of_crimes DESC")

#display result
display(crimesSFdt.take(20))


Date,num_of_crimes
01/01/2006,14227
04/13/2014,12163
05/25/2014,10763
11/23/2014,9488
11/24/2013,9212
01/16/2011,9116
04/21/2013,9109
10/27/2013,8890
08/03/2014,8874
08/24/2003,8771


#### Q3. Conclustion:
According to calculations, we can conclude that amongst all Sundays, on 01/01/2006 the number of crimes is at its highest level.
01/01/2006 appeared to be during New Year Holidays, and this could be a major reason which resulted in a high number of crimes.
Aadditional information needed.

### Q4 (OLAP)
Analysis the number of crime in each month of 2015, 2016, 2017, 2018. Then, give your insights for the output results. What is the business impact for the result?

In [22]:
#Spark SQL based - Soulution for Q4
#counts the number of crimes in each month of 2015, 2016, 2017, 2018

#format the date MM/dd/yyyyy to YYYY-MM-DD 
crimeInYearMonth = spark.sql("SELECT year(to_date(date, 'MM/dd/yyyy')) as year, \
                                     month(to_date(date, 'MM/dd/yyyy')) as month, \
                                     COUNT(*) as num_of_crimes \
                              FROM sf_crime \
                              GROUP BY year, month \
                              HAVING year in (2015, 2016, 2017, 2018) \
                              ORDER BY num_of_crimes DESC")
                             
#display result
display(crimeInYearMonth.take(20))


#using extract() function:
#crimeInYearMonth = spark.sql("SELECT EXTRACT(YEAR FROM dt.date) as year, \
                              #       EXTRACT(MONTH FROM dt.date) AS month, \
                              #       COUNT(*) as num_of_crimes \
                              #FROM (\
                              #       SELECT TO_DATE(CAST(UNIX_TIMESTAMP(date, 'MM/dd/yyyy') AS TIMESTAMP)) AS date \
                              #       FROM sf_crime \
                              #     ) AS dt \
                              #GROUP BY year, month \
                              #HAVING year in (2015, 2016, 2017, 2018) \
                              #ORDER BY num_of_crimes DESC")
                  

year,month,num_of_crimes
2015,3,13929
2015,8,13730
2015,5,13729
2017,3,13711
2015,1,13606
2016,10,13388
2015,7,13365
2017,10,13355
2015,6,13304
2017,5,13267


#### Q4. Conclution:
The number of crimes hit its highest level in March 2015:
looking at the data we obtained from 2015-2018, we can also conclude that the year 2015 has the most crime cases becasue the top-3 months are all in 2015.

#### Business impact for the result:
There might be some incidents happened in 2015 (politically or financially).
Therefore, for companies in SF, they might have been impacted to some extent.
In these particular months and years, business activities (in the sector of goods and services) might have been slightly affected as well due to safety concerns expressed by people.

### Q5 (OLAP)
Analysis the number of crime w.r.t the hour in certian day like 2015/12/15, 2016/12/15, 2017/12/15. Then, give travel suggestion to visit SF.

In [25]:
#Spark SQL based - Soulution for Q4
#counts the number of crimes in each month of 2015, 2016, 2017, 2018

crimesDate = spark.sql("SELECT res.date, res.hours, COUNT(*) as num_of_crimes \
                        FROM ( \
                               SELECT date, \
                                      CASE WHEN time BETWEEN '00:00' AND '06:00:00' THEN 'early morning' \
                                           WHEN time BETWEEN '06:00' AND '12:00:00' THEN 'morning' \
                                           WHEN time BETWEEN '12:00' AND '18:00:00' THEN 'afternoon' \
                                           WHEN time BETWEEN '18:00' AND '24:00:00' THEN 'night' \
                                      END AS hours \
                               FROM sf_crime \
                               WHERE date IN ('12/15/2015', '12/15/2016', '12/15/2017') \
                        ) AS res \
                        GROUP BY 1,2 \
                        ORDER BY num_of_crimes DESC")

#display result
display(crimesDate.take(20))


date,hours,num_of_crimes
12/15/2017,night,154
12/15/2016,night,135
12/15/2017,afternoon,133
12/15/2015,afternoon,124
12/15/2017,morning,118
12/15/2016,afternoon,116
12/15/2016,morning,99
12/15/2015,morning,93
12/15/2015,night,88
12/15/2017,early morning,66


#### Q5. Conclustion:
Analyzed the number of crime with respect to the hours in 12/15/2015, 12/15/2016; 12/15/2017: between 00:00-6:00 'early morning'; between 6:00-12:00 'morning'; between 12:00-18:00 'afternoon'; and lastly between 18:00-00:00 'night'.

#### Travel tips to visit SF:
In general, afternoon and night (between 12:00 and 00:00) have higher crime rates, whereas early morning and morning hours appear to be much safer.
My advice would be if you plan to visit San Francisco, try not to arrive at SF city later than 12PM.
Instead, you should prepare one day earlier and set out in the early morning or in the morning,
and that so after you chcek-in your hotel, you can relax a bit and explore the city during daytimes.
Remember to go back home early (no later than 18:00), because there are more chances to be involed in crimes in SF at night.

### Q6 (OLAP)
(1) Step1: Find out the top-3 danger disrict  
(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  
(3) give your advice to distribute the police based on your analysis results.

In [28]:
#(1) Step1: Find out the top-3 danger disrict  
#crimePdDistrict_desc = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count DESC")
#display(crimePdDistrict_desc.take(3))

#Top-3 danger district: Southern, Mission, and Norther


#(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  

##three districts were calculated respectively:
crimes_in_top3 = spark.sql("SELECT res.category, \
                                   res.PdDistrict, \
                                   res.hours, \
                                   COUNT(*) as num_of_crimes \
                            FROM ( \
                                  SELECT category, PdDistrict, \
                                         CASE WHEN time BETWEEN '00:00' AND '06:00:00' THEN 'early morning' \
                                              WHEN time BETWEEN '06:00' AND '12:00:00' THEN 'morning' \
                                              WHEN time BETWEEN '12:00' AND '18:00:00' THEN 'afternoon' \
                                              WHEN time BETWEEN '18:00' AND '24:00:00' THEN 'night' \
                                         END AS hours \
                                  FROM sf_crime \
                                  WHERE PdDistrict IN ('SOUTHERN', 'MISSION', 'NORTHERN') \
                             ) AS res \
                             GROUP BY 1,2,3 \
                             ORDER BY num_of_crimes DESC")

#display result
display(crimes_in_top3.take(20))

#(3) give your advice to distribute the police based on your analysis results. 
##from the results we obtained, since we are looking at the top-3 danger districts, police force should be distributed primarily in Southern district, then Nothern district followed by Mission district
##police on-duty time is suggested to be at night for LARCENY/THEFT; in the afternoon for OTHER OFFENSES
##residents in the city should cooperate with the police and stay at home during dangerous hours from 12:PM to midnight

category,PdDistrict,hours,num_of_crimes
LARCENY/THEFT,SOUTHERN,afternoon,41261
LARCENY/THEFT,SOUTHERN,night,39937
LARCENY/THEFT,NORTHERN,night,30390
LARCENY/THEFT,NORTHERN,afternoon,26616
LARCENY/THEFT,SOUTHERN,morning,21001
LARCENY/THEFT,MISSION,night,18147
NON-CRIMINAL,SOUTHERN,afternoon,17250
OTHER OFFENSES,SOUTHERN,afternoon,17156
OTHER OFFENSES,MISSION,afternoon,15244
LARCENY/THEFT,MISSION,afternoon,15153


### Q7 (OLAP)
For different category of crime, find the percentage of resolution. Based on the output, give hints to adjust the policy.

In [30]:
#write subqueries tb1 & tb2 to obtain results where resolution is None("Unresolved") or else("resolved")
#left join two tables together
#calculate percentage of resolution
#cnt_un/(cnt_un+cnt_re) is the percentage of unresolved crimes, saved as cnt_un_pc
#cnt_re/(cnt_un+cnt_re) is the percentage of resolved crimes, saved as cnt_re_pc

crimeCategoryPercentage = spark.sql("SELECT tb3.Category, \
                                            cnt_un/(cnt_un+cnt_re) as cnt_un_pc, \
                                            cnt_re/(cnt_un+cnt_re) as cnt_re_pc \
                                     FROM ( \
                                           SELECT tb1.Category, cnt_un, cnt_re \
                                           FROM( \
                                                SELECT Category, COUNT(resolved_or_not) as cnt_un \
                                                FROM (\
                                                      SELECT Category, \
                                                             CASE WHEN Resolution LIKE 'NONE' THEN 'unresolved' \
                                                             ELSE 'resolved' \
                                                             END AS resolved_or_not \
                                                      FROM sf_crime) \
                                                      WHERE resolved_or_not = 'unresolved' \
                                                      GROUP BY Category \
                                                 ) tb1 \
                                           LEFT JOIN \
                                           (SELECT Category, COUNT(resolved_or_not) as cnt_re \
                                            FROM ( \
                                                  SELECT Category, \
                                                         CASE WHEN Resolution LIKE 'NONE' THEN 'unresolved' \
                                                         ELSE'resolved' \
                                                         END AS resolved_or_not \
                                                  FROM sf_crime) \
                                                  WHERE resolved_or_not = 'resolved' \
                                                  GROUP BY Category \
                                           ) tb2 \
                                           ON tb1.Category = tb2.Category \
                                    ) tb3 \
                                    ORDER BY cnt_un_pc DESC")
 
#display result
display(crimeCategoryPercentage.take(20))


Category,cnt_un_pc,cnt_re_pc
RECOVERED VEHICLE,0.9308168884809546,0.06918311151904544
VEHICLE THEFT,0.916099271733464,0.08390072826653608
LARCENY/THEFT,0.9114971859597709,0.08850281404022912
SUSPICIOUS OCC,0.8824275272239073,0.11757247277609269
VANDALISM,0.8779155429565997,0.12208445704340035
BURGLARY,0.8373441989010629,0.16265580109893713
ARSON,0.8074281353345205,0.19257186466547951
BAD CHECKS,0.8043243243243243,0.19567567567567568
EXTORTION,0.7786774628879892,0.2213225371120108
NON-CRIMINAL,0.7756616021114202,0.2243383978885798


#### Q7. Hints to adjust the policy:
1) Police officers should focus more on cimes like RECOVERED VEHICLE, VEHICLE THEFT, and LARCENY/THEFT, because this three categories have the highest 'unresolved' percentages.

2) Crimes related to vehicles require additional attention from the police department.

3) Police should take some actions to raise the awareness of vehicle owners, as a result of common vehicles safty/security issues.

### Conclusion. 
Use four sentences to summary your work. Like what you have done, how to do it, what the techinical steps, what is your business impact. 
More details are appreciated. You can think about this a report for your manager. Then, you need to use this experience to prove that you have strong background on big  data analysis.  
Point 1:  what is your story ? and why you do this work ?   
Point 2:  how can you do it ?  keywords: Spark, Spark SQL, Dataframe, Data clean, Data visulization, Data size, clustering, OLAP,   
Point 3:  what do you learn from the data ?  keywords: crime, trend, advising, conclusion, runtime

#### HW1 Conclusion:
I have leveraged a large-sized data set of SF crimes using Spark, Spark SQL in Databricks.
The goal was to provide some safety suggestions (travel tips) to residents who live in SF, and to travelers who plan to visit the city.
Prior to jumping into the homework, we need to create a clustering to actualize code implementation.
I did some data cleaning as well as data processing through importing packages and through using lines of SQL code, such as applying map() & filter() functions.
In this homework, I mainly used spark SQL to answer OLAP questions.
Some useful findings include SF top-3 common crimes, SF top-3 danger districts, number of crimes around downtown SF, hours (time) for particular dates, and resolved & unresolved percentages for different crime categories, etc.

Advertisings on safety topics are necessarily needed to raise awareness primarily among residents in SOUTHERN, MISSION, and NORTHERN districts.
Furthermore, police officers should focus more on cimes like RECOVERED VEHICLE, VEHICLE THEFT, and LARCENY/THEFT, because this three categories have the highest 'unresolved' percentage.
Runtime for question 3 is relatively long, because the result (dataframe) we want to obtain is large.

Overall, my suggestion would be avoiding visiting the city or walking around the streets at night or in the afternoon,
because the chances of being involved in a crime are higher than that during other hours.
In addition, pay extra attention during holidays, because during these time, crimes tend to be higher (for example, see Q3. -> 01/01/2006 Sunday num_of_crimes).